# PredictIt Arbitrage Finder
Takes a look at every market on PredictIt and looks for best arbitrage opportunities.

In [1]:
import mechanicalsoup
import pandas as pd
import numpy as np
import json
import datetime
import math

browser = mechanicalsoup.Browser()
data=browser.get('https://www.predictit.org/api/marketdata/all/').json()

In [2]:
#set up dataframe
marketdf=pd.DataFrame(data)
df=pd.json_normalize(marketdf['markets'])

In [3]:
presidentdf=df[df['shortName'].str.startswith('Which party will win') & (df['shortName'].str.endswith('in 2020?') | df['shortName'].str.endswith(' 2020?'))]
df[df['name'].str.contains('Wyoming')]

,id,name,shortName,image,url,contracts,timeStamp,status
115,6632,Which party will win Wyoming in the 2020 presi...,Which party will win WY 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6632/...,"[{'id': 22241, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open
162,6797,Which party will win the U.S. Senate election ...,Which party will win the WY Senate race?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6797/...,"[{'id': 23237, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open


In [4]:
presidentdf=presidentdf[(presidentdf['shortName'].str.len()<=32)]
presidentdf['shortName']

31     Which party will win WI in 2020?
32     Which party will win PA in 2020?
33     Which party will win FL in 2020?
34     Which party will win MI in 2020?
36     Which party will win AZ in 2020?
37     Which party will win MN in 2020?
38     Which party will win NH in 2020?
39     Which party will win NC in 2020?
40     Which party will win OH in 2020?
41     Which party will win NV in 2020?
42     Which party will win VA in 2020?
43     Which party will win IA in 2020?
44     Which party will win GA in 2020?
45     Which party will win CO in 2020?
49     Which party will win TX in 2020?
82     Which party will win ME in 2020?
83     Which party will win IN in 2020?
84     Which party will win NM in 2020?
87     Which party will win NJ in 2020?
88     Which party will win MO in 2020?
89     Which party will win OR in 2020?
90     Which party will win UT in 2020?
91     Which party will win TN in 2020?
92     Which party will win CT in 2020?
93     Which party will win AK in 2020?


In [5]:
presidentdf['state']=presidentdf['shortName'].str[21:23]

In [6]:
fivethirtyeight=pd.read_csv('https://projects.fivethirtyeight.com/2020-general-data/presidential_state_toplines_2020.csv')

In [7]:
fivethirtyeight.columns

Index(['cycle', 'branch', 'model', 'modeldate', 'candidate_inc',
       'candidate_chal', 'candidate_3rd', 'state', 'tipping', 'vpi',
       'winstate_inc', 'winstate_chal', 'winstate_3rd', 'voteshare_inc',
       'voteshare_chal', 'voteshare_3rd', 'voteshare_other',
       'voteshare_inc_hi', 'voteshare_chal_hi', 'voteshare_3rd_hi',
       'voteshare_other_hi', 'voteshare_inc_lo', 'voteshare_chal_lo',
       'voteshare_3rd_lo', 'voteshare_other_lo', 'margin', 'margin_hi',
       'margin_lo', 'win_EC_if_win_state_inc', 'win_EC_if_win_state_chal',
       'win_state_if_win_EC_inc', 'win_state_if_win_EC_chal', 'state_turnout',
       'state_turnout_hi', 'state_turnout_lo', 'timestamp', 'simulations'],
      dtype='object')

In [8]:
fivethirtyeight=fivethirtyeight[fivethirtyeight['modeldate']==datetime.date.strftime(datetime.date.today(), "%m/%d/%Y")]

In [9]:
states=pd.read_csv('name-abbr.csv')

In [10]:
states

,Name,Abbrev
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District Of Columbia,DC
9,Florida,FL


In [11]:
alldf=pd.merge(pd.merge(presidentdf,states,how="outer",left_on='state',right_on='Abbrev'),fivethirtyeight,left_on='Name',right_on='state')

In [12]:
def rounddown(a,places=2):
    a = math.floor(a * pow(10,places)) / pow(10,places)
    return a

alldf['repubprob']=alldf['winstate_inc'].apply(rounddown)
alldf['demprob']=alldf['winstate_chal'].apply(rounddown)

In [14]:
pd.json_normalize(alldf['contracts'][1])

,id,dateEnd,image,name,shortName,status,lastTradePrice,bestBuyYesCost,bestBuyNoCost,bestSellYesCost,bestSellNoCost,lastClosePrice,displayOrder
0,15705,N/A,https://az620379.vo.msecnd.net/images/Contract...,Democratic,Democratic,Open,0.65,0.66,0.35,0.65,0.34,0.65,0
1,15704,N/A,https://az620379.vo.msecnd.net/images/Contract...,Republican,Republican,Open,0.37,0.37,0.64,0.36,0.63,0.38,0


In [15]:
alldf['dempriceNO']=0.0
alldf['reppriceNO']=0.0

In [16]:
#Get two way arb
for i in range(0,len(alldf)):
    print(i)
    contracts=pd.json_normalize(alldf['contracts'][i])
    demprice = contracts[contracts['name']=='Democratic']['bestBuyNoCost']
    repprice = contracts[contracts['name']=='Republican']['bestBuyNoCost']
    alldf['dempriceNO'][i] = demprice
    alldf['reppriceNO'][i] = repprice

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [17]:
alldf['kellyRepNo']=alldf['demprob']-(1-alldf['demprob'])/(.9*(1-alldf['reppriceNO'])/alldf['reppriceNO'])
alldf['kellyDemNo']=alldf['repubprob']-(1-alldf['repubprob'])/(.9*(1-alldf['dempriceNO'])/alldf['dempriceNO'])

In [22]:
alldf.sort_values('kellyRepNo',ascending=False).head(10)

,id,name,shortName,image,url,contracts,timeStamp,status,state_x,Name,...,state_turnout_hi,state_turnout_lo,timestamp,simulations,repubprob,demprob,dempriceNO,reppriceNO,kellyRepNo,kellyDemNo
10,5602,Which party will win Virginia in the 2020 pres...,Which party will win VA in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/5602/...,"[{'id': 16012, 'dateEnd': 'N/A', 'image': 'htt...",2020-10-21T12:07:57.8734341,Open,VA,Virginia,...,4848092.0,3740613.0,11:06:05 21 Oct 2020,40000,0.0,0.99,0.10,0.90,0.890000,-0.123457
33,6612,Which party will win New York in the 2020 pres...,Which party will win NY in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6612/...,"[{'id': 22162, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,NY,New York,...,8899502.0,6772883.0,11:06:05 21 Oct 2020,40000,0.0,0.99,0.08,0.92,0.862222,-0.096618
32,6611,Which party will win California in the 2020 pr...,Which party will win CA in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6611/...,"[{'id': 22160, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,CA,California,...,16500000.0,12500000.0,11:06:05 21 Oct 2020,40000,0.0,0.99,0.07,0.92,0.862222,-0.083632
29,6598,Which party will win Washington in the 2020 pr...,Which party will win WA in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6598/...,"[{'id': 22114, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,WA,Washington,...,4130124.0,3164438.0,11:06:05 21 Oct 2020,40000,0.0,0.99,0.07,0.92,0.862222,-0.083632
44,6631,Which party will win Hawaii in the 2020 presid...,Which party will win HI in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6631/...,"[{'id': 22240, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,HI,Hawaii,...,501627.5,375376.0,11:06:05 21 Oct 2020,40000,0.0,0.99,0.07,0.93,0.842381,-0.083632
34,6613,Which party will win Illinois in the 2020 pres...,Which party will win IL in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6613/...,"[{'id': 22164, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,IL,Illinois,...,6458305.0,4960260.0,11:06:05 21 Oct 2020,40000,0.0,0.99,0.07,0.93,0.842381,-0.083632
47,6636,Which party will win Delaware in the 2020 pres...,Which party will win DE in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6636/...,"[{'id': 22264, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,DE,Delaware,...,528663.0,403068.0,11:06:05 21 Oct 2020,40000,0.0,0.99,0.07,0.94,0.815926,-0.083632
46,6633,Which party will win Vermont in the 2020 presi...,Which party will win VT in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6633/...,"[{'id': 22244, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,VT,Vermont,...,368200.0,277264.5,11:06:05 21 Oct 2020,40000,0.0,0.99,0.07,0.94,0.815926,-0.083632
27,6596,Which party will win Massachusetts in the 2020...,Which party will win MA in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6596/...,"[{'id': 22110, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,MA,Massachusetts,...,3862282.0,2934019.0,11:06:05 21 Oct 2020,40000,0.0,0.99,0.07,0.94,0.815926,-0.083632
23,6587,Which party will win Connecticut in the 2020 p...,Which party will win CT in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6587/...,"[{'id': 22081, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,CT,Connecticut,...,1881711.0,1450092.0,11:06:05 21 Oct 2020,40000,0.0,0.99,0.07,0.94,0.815926,-0.083632


In [21]:
alldf.sort_values('kellyDemNo',ascending=False).head(4)

,id,name,shortName,image,url,contracts,timeStamp,status,state_x,Name,...,state_turnout_hi,state_turnout_lo,timestamp,simulations,repubprob,demprob,dempriceNO,reppriceNO,kellyRepNo,kellyDemNo
39,6624,Which party will win Nebraska in the 2020 pres...,Which party will win NE in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6624/...,"[{'id': 22220, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,NE,Nebraska,...,998045.0,753777.0,11:06:05 21 Oct 2020,40000,0.99,0.00,0.95,0.06,-0.070922,0.778889
28,6597,Which party will win Arkansas in the 2020 pres...,Which party will win AR in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6597/...,"[{'id': 22111, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,AR,Arkansas,...,1321178.0,994043.0,11:06:05 21 Oct 2020,40000,0.98,0.01,0.94,0.07,-0.072796,0.631852
45,6632,Which party will win Wyoming in the 2020 presi...,Which party will win WY 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6632/...,"[{'id': 22241, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,WY,Wyoming,...,294408.0,212729.0,11:06:05 21 Oct 2020,40000,0.99,0.00,0.97,0.04,-0.046296,0.630741
38,6623,Which party will win Idaho in the 2020 preside...,Which party will win ID in 2020?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6623/...,"[{'id': 22218, 'dateEnd': 'NA', 'image': 'http...",2020-10-21T12:07:57.8734341,Open,ID,Idaho,...,861292.5,637845.0,11:06:05 21 Oct 2020,40000,0.99,0.00,0.97,0.04,-0.046296,0.630741
